<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/OracleKevinBacon/OracleKevinBacon_v_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import zipfile
import pandas as pd

In [2]:
!gdown --id 1nQOU1QjZJNnCpnihSMbuTkpUvMzpAVfH

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1nQOU1QjZJNnCpnihSMbuTkpUvMzpAVfH
To: /content/baconOracle.zip
100% 59.1M/59.1M [00:00<00:00, 210MB/s]


In [3]:
file_name = zipfile.ZipFile('baconOracle.zip', 'r')
file_name.extractall()

In [4]:
dataset_name ='cast_data.csv'
df1 = pd.read_csv(dataset_name)
df1.columns = ['person_id','name','birth']
df1.head()

,person_id,name,birth
0,1,Fred Astaire,1899.0
1,2,Lauren Bacall,1924.0
2,3,Brigitte Bardot,1934.0
3,4,John Belushi,1949.0
4,5,Ingmar Bergman,1918.0


In [5]:
dataset_name ='cast_and_movies_data.csv'
df2 = pd.read_csv(dataset_name)
df2.head()

,person_id,movie_id
0,844752,15724
1,869732,15724
2,194720,15724
3,650495,15724
4,8738,31458


In [6]:
dataset_name ='movies_data.csv'
df3 = pd.read_csv(dataset_name)
df3.columns = ['movie_id','title','year']
df3.head()

,movie_id,title,year
0,15724,Dama de noche,1993
1,23331,Pesn o geroyakh,1983
2,31458,El huésped del sevillano,1970
3,35423,Kate & Leopold,2001
4,36606,"Another Time, Another Place",1983


In [7]:
merged = pd.merge(df1, df2, on='person_id')
merged.head()

,person_id,name,birth,movie_id
0,1,Fred Astaire,1899.0,72272
1,1,Fred Astaire,1899.0,74130
2,1,Fred Astaire,1899.0,75323
3,1,Fred Astaire,1899.0,76851
4,1,Fred Astaire,1899.0,82449


In [8]:
merged = pd.merge(merged, df3, on='movie_id')
merged.head()

,person_id,name,birth,movie_id,title,year
0,1,Fred Astaire,1899.0,72272,That's Entertainment!,1974
1,37,Gene Kelly,1912.0,72272,That's Entertainment!,1974
2,1078,Bing Crosby,1903.0,72272,That's Entertainment!,1974
3,492444,Peter Lawford,1923.0,72272,That's Entertainment!,1974
4,1,Fred Astaire,1899.0,74130,The Amazing Dobermans,1976


In [9]:
movies = set(merged['title'].values.tolist())

In [10]:
cont = 0

In [11]:
network={}
for k in movies:
  cont = cont + 1
  if(cont % 10 == 0):
    print('feito',cont,'de',Num)
  Name =k
  try:
    df=merged[merged['title'].str.contains(Name, na=False)]
    cast = df.name.values.tolist()
    network[Name]=cast
  except:
    pass



feito 10 de 256002
feito 20 de 256002
feito 30 de 256002
feito 40 de 256002
feito 50 de 256002
feito 60 de 256002
feito 70 de 256002
feito 80 de 256002
feito 90 de 256002
feito 100 de 256002
feito 110 de 256002
feito 120 de 256002
feito 130 de 256002
feito 140 de 256002
feito 150 de 256002
feito 160 de 256002
feito 170 de 256002
feito 180 de 256002
feito 190 de 256002
feito 200 de 256002


In [12]:

imdb = network

def getCast(movie):
    global imdb
    return imdb[movie]

def getFilmography(actor):
    global imdb
    s = []
    for movie, cast in imdb.items():
        if actor in cast:
            s.append(movie)
    return s

def init_global_data():
    global actorList
    global adjacencyMatrix
    global cache

    # Get the list of all actors.
    actorSet = set()
    for cast in imdb.values():
        actorSet.update(cast)
    actorList = list(actorSet)
    N = len(actorList)

    # Initialize the adjacency matrix as an N by N matrix of zeroes.
    adjacencyMatrix = np.array([0] * (N * N)).reshape(N, N)

    # Now populate it.
    for i, actor in enumerate(actorList):
        for movie in getFilmography(actor):
            for costar in getCast(movie):
                j = actorList.index(costar)
                adjacencyMatrix[i,j] += 1

    # Initialize the cache.
    cache = [None, adjacencyMatrix]

def get_power(N):
    """
        Get the Nth power of the adjacency matrix.
        Cache the result to minimize the number of
        matrix multiplications we have to perform.
    """
    global adjacencyMatrix
    global cache

    while len(cache) < N + 1:
        # Get a reference to the highest power we have calculated so far.
        # Calculate the next power and add the matrix to the cache.
        cache.append(np.dot(cache[-1], adjacencyMatrix))

    # Return a reference to a matrix in the cache.
    return cache[N]

def actorNumber(actor, referenceActor):
    """
        Calculate an actor's number relative to a reference actor.
    """
    global actorList
    i = actorList.index(referenceActor)
    j = actorList.index(actor)
    if i == j:
        return 0
    power = 1
    A = get_power(power)
    while A[i,j] <= 0:
        power += 1
        if power > 10:
            # Quick and dirty check against hanging.
            raise Exception("Infinite loop")
        A = get_power(power)
    return power

def KevinBaconNumber(actor):
    """
        Calculate the actor's Kevin Bacon number.
    """
    return actorNumber(actor, 'Kevin Bacon')

if __name__ == "__main__":
    init_global_data()
    actor ='Michael Douglas'
    print(actor, KevinBaconNumber(actor)) 

ValueError: ignored